# Vytváření Python rozharaní pro C kód v Pythonu
Python poskytuje hlavičkové soubory pro C, které umožňují přistupovat k Python objektům. 
Pomocí těchto hlavičkových souborů lze vytvářet C knihovnu, které se pro Python bude tvářit jako nativní Python knihovna.

Ukážeme si malou ukázku, více v dokumentaci [zde](https://docs.python.org/3/c-api/).

### Parsování vstupu z Pythonu do C

Funkce která bude volána z Pythonu bude v C vypadat nějak takto:

```c
static PyObject* moje_fce(PyObject *self, PyObject *args) 
```

Tedy všechny vstupy z Pythonu budou předány jako `PyObject *args`. A očekává se návratová hodnota `PyObject*`.

Proto prvním hlavním stavebním kamenem C API je parsování vstupu z Pythonu do C.

Funkce `PyArg_ParseTuple` je součástí Python C API a slouží k parsování argumentů předaných z Pythonu do funkce v jazyce C. Umožňuje konverzi Python objektů předaných jako argumenty do vaší C funkce na hodnoty v jazyce C.

Signatura funkce je následující:
`int PyArg_ParseTuple(PyObject *args, const char *format, ...);`

**Parametry**

1.  `args`: Ukazatel na Python tuple objekt obsahující argumenty předané z Pythonu. (Tedy to co přišlo jako `*args`)
2.  `format`: Formátovací řetězec, který specifikuje očekávané typy a počet argumentů v tuple. Každý znak ve formátovacím řetězci odpovídá očekávanému typu argumentu.
    *   `s`: Řetězec (ukončený nulou)
    *   `i`: Celé číslo
    *   `f`: Číslo s plovoucí řádovou čárkou
    *   `d`: Dvojitá přesnost (double)
    *   `O`: Python objekt (obecný)
    *   `|`: Udává, že následující argumenty jsou volitelné
    *   `O!`: Python objekt specifického typu, následovaný ukazatelem na Python typový objekt
3.  `...`: Proměnný počet ukazatelů na proměnné v jazyce C, které budou obsahovat převedené hodnoty argumentů. Tyto ukazatele by měly odpovídat počtu a pořadí specifikátorů formátu ve formátovacím řetězci.



### Vytváření obektů/struktur viditelných pro Python
#### Metody
`PyMethodDef` je struktura v Python C API, která definuje metody modulu, obsahuje následující záznamy:

1. Název metody jako řetězec (např. "my_c_function").
2. Ukazatel na C funkci, která implementuje metodu (např. `my_c_function`).
3. Příznak způsobu předání argumentů (`METH_VARARGS` pro proměnný počet argumentů ve formě n-tice).
4. Řetězec s dokumentací metody ("Multiply all elements of a NumPy array by 2").

Seznam metod je pak pole těchto struktur, které uvádí metody, které budou v modulu dostupné. 

Poslední prvek pole musí obsahovat nuly a NULL ukazatele, které slouží jako ukončovací značka pro pole metod.


```c
static PyMethodDef MyCModuleMethods[] = {
    {"my_c_function", my_c_function, METH_VARARGS, "Multiply all elements of a NumPy array by 2"},
    {NULL, NULL, 0, NULL}
};
```

Při vytváření C modulu pro Python je toto pole metod předáno do funkce `PyModule_Create`, která vytvoří modul s definovanými metodami.


#### Modul
Informace o modulu definuje struktura `PyModuleDef` z Python C API. Skládá se z:

1. `PyModuleDef_HEAD_INIT`: Makro pro inicializaci hlavičky struktury modulu.
2. Název modulu jako řetězec (např. "my_c_module").
3. Ukazatel na řetězec s dokumentací modulu (v tomto případě `NULL`, protože dokumentace není poskytnuta).
4. Velikost prostoru modulu (`-1` značí, že modul nemá žádný stav a všechny jeho funkce jsou globální).
5. Ukazatel na pole metod definovaných v modulu (viz stuktura `PyMethodDef`).


```c
static struct PyModuleDef my_c_module = {
    PyModuleDef_HEAD_INIT,
    "my_c_module",
    NULL,
    -1,
    MyCModuleMethods
};
```

Tato struktura modulu je poté předána do funkce `PyModule_Create`, která vytvoří modul s definovanými metodami a případným stavem.

#### Inicializace modulu
Pro nás poslední "povinnou" částí je inicializace modulu ať skutečně vše co jsme doposud definovali (zatím jenom struktury v C) je viditelné pro Python.

Funkce `PyMODINIT_FUNC PyInit_my_c_module(void)` je inicializační funkce modulu, která se volá, když Python načítá váš C modul. Tato funkce má následující úkoly:

1. Vytvořit modul pomocí funkce `PyModule_Create`, která přijímá ukazatel na strukturu `PyModuleDef` (v tomto případě `&my_c_module`).
2. Zkontrolovat, zda se modul úspěšně vytvořil. Pokud `module` je `NULL`, vrátit `NULL` a ukončit inicializaci.
3. Inicializovat externí závislosti, pokud existují. V tomto příkladu se volá funkce `import_array()` pro inicializaci NumPy C API.
4. Vrátit vytvořený modul, který bude následně zaregistrován v Pythonu.

```c
PyMODINIT_FUNC PyInit_my_c_module(void) {
    PyObject* module = PyModule_Create(&my_c_module);
    if (module == NULL) {
        return NULL;
    }

    // Initialize the NumPy C API
    import_array();

    return module;
}
```

Tato inicializační funkce musí vracet `PyMODINIT_FUNC` a musí mít název `PyInit_` následovaný názvem modulu.

In [ ]:
%%writefile mvc_mylib.c

#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION

#include <Python.h>
#include <numpy/arrayobject.h>
#include "matrix_vector_multiply.c"

static PyObject* mvp_func_py(PyObject *self, PyObject *args) {
    // Declare variables
    PyArrayObject *matrix, *vector, *result;
    int rows, cols;

    if (!PyArg_ParseTuple(args, "O!O!O!", &PyArray_Type, &matrix, &PyArray_Type, &vector, &PyArray_Type, &result)) {
        return NULL;
    }

    rows = (int)PyArray_DIM(matrix, 0);
    cols = (int)PyArray_DIM(matrix, 1);

    matrix_vector_multiply((float *)PyArray_DATA(matrix), (float *)PyArray_DATA(vector), (float *)PyArray_DATA(result), rows, cols);

    Py_RETURN_NONE;
};

static PyMethodDef MatrixVectorMultiplyMethods[] = {
    {"mvp_func_py", mvp_func_py, METH_VARARGS, "Perform matrix-vector multiplication."},
    {NULL, NULL, 0, NULL}
};

static struct PyModuleDef matrix_vector_multiply_module = {
    PyModuleDef_HEAD_INIT,
    "mvc_mylib",
    NULL,
    -1,
    MatrixVectorMultiplyMethods
};

PyMODINIT_FUNC PyInit_mvc_mylib(void) {
    PyObject* module = PyModule_Create(&matrix_vector_multiply_module);
    import_array();
    return module;
};

In [ ]:
%%writefile setup.py
from setuptools import setup, Extension
import numpy as np

matrix_vector_multiply_module = Extension(
    'mvc_mylib',
    sources=['mvc_mylib.c'],
    include_dirs=[np.get_include()],
)

setup(
    name='mvc_mylib',
    version='1.0',
    description='A C extension for multiplying a matrix and a vector',
    ext_modules=[matrix_vector_multiply_module],
)


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import numpy as np
import mvc_mylib


In [ ]:
matrix = np.array([[1, 2, 3, 4],
                   [5, 6, 7, 8],
                   [9, 10, 11, 12]], dtype=np.float32)

vector = np.array([2, 1, 3, 0], dtype=np.float32)
result = np.zeros((3), dtype=np.float32)

mvc_mylib.mvp_func_py(matrix, vector, result)

print("Result:")
print(result)

In [ ]:
matrix@vector